In [1]:
import torch
import requests

from bs4 import BeautifulSoup
from transformers import T5Tokenizer,T5ForConditionalGeneration

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)
tokenizer = T5Tokenizer.from_pretrained('t5-base')

def preprocess_text(url):
    response = requests.get(url)
    content = response.content
    soup_article = BeautifulSoup(content, "html5lib")
    body = soup_article.find_all("div", class_ = None)
    s = ""
    for i in body:
        s+=i.text
    return s

def summarize(text):
    t5_prep = "summarize: " + str(text).strip().replace("\n", "")
    tokenized_text = tokenizer.encode(t5_prep, max_length = len(t5_prep), return_tensors = "pt", truncation = True).to(device)
    summary_ids = model.generate(tokenized_text, num_beams = 4,
                                 no_repeat_ngram_size = 2,
                                 min_length = 120,
                                 max_length = 350,
                                 early_stopping = True)
    output = tokenizer.decode(summary_ids[0].to(device), skip_special_tokens = True)
    return output

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
url = "https://timesofindia.indiatimes.com/india/lac-stand-off-india-china-hope-to-pull-back-from-brink-in-likely-meeting/articleshow/78027119.cms"

In [6]:
summarize(preprocess_text(url))

'India and China will look to pull back from the brink of military conflict in a likely meeting between foreign minister S Jaishankar and his Chinese counterpart Wang Yi on the margins of SCO in Moscow Thursday. while the Chinese foreign ministry didn\'t confirm the meeting, Chinese sources here said it was expected, even if there was "no confirmed information" the trilateral could help break the ice ahead of what will be the first in-person bilateral meeting in India after the ongoing military standoff was reported in May.\'- gragra '